In [1]:
import pandas as pd
import json


def load_json_to_df(file_path):
    data = []
    with open(file_path, 'r') as file:
        for line in file:
            data.append(json.loads(line))
    return pd.DataFrame(data)

# Paths to the JSON files in the Kaggle environment

business_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json'
checkin_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_checkin.json'
review_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json'
tip_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_tip.json'
user_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json'

# Load the datasets
business_df = load_json_to_df(business_path)
checkin_df = load_json_to_df(checkin_path)
review_df = load_json_to_df(review_path)
tip_df = load_json_to_df(tip_path)
user_df = load_json_to_df(user_path)


checkin_df['date'] = checkin_df['date'].str.split(', ')
checkin_df = checkin_df.explode('date')


review_with_business = review_df.merge(business_df[['business_id', 'city']], on='business_id', how='left')


checkin_with_business = checkin_df.merge(business_df[['business_id', 'city']], on='business_id', how='left')


# Query 1: Select all businesses in the state of Illinois (IL)
business_in_il = business_df[business_df['state'] == 'IL']

# Query 2: Select all businesses with a star rating greater than 4
business_above_4_stars = business_df[business_df['stars'] > 4]

# Query 3: Select all 5-star businesses in Illinois (IL)
five_star_business_in_il = business_df[(business_df['stars'] == 5) & (business_df['state'] == 'IL')]

# Query 4: Select top 10 businesses in New Orleans by review count
top_business_in_new_orleans = business_df[business_df['city'] == 'New Orleans'].nlargest(10, 'review_count')

# Query 5: Select distinct categories of 5-star businesses in Illinois (IL)
distinct_categories_in_il = five_star_business_in_il['categories'].unique()

# Query 6: Select top 5 businesses in New Orleans with more than 100 reviews, ordered by stars descending
top_business_over_100_reviews = business_df[
    (business_df['city'] == 'New Orleans') & (business_df['review_count'] > 100)
].nlargest(5, 'stars')

# Query 7: Select top 5 categories in New Orleans with more than 4.5 stars
top_categories_over_45_stars = business_df[
    (business_df['city'] == 'New Orleans') & (business_df['stars'] > 4.5)
]['categories'].value_counts().head(5)

# Query 8: Select average review count per category in New Orleans
average_review_count_new_orleans = business_df[
    business_df['city'] == 'New Orleans'
].groupby('categories')['review_count'].mean().sort_values(ascending=False)

# Query 9: Select average star rating and number of reviews for businesses in New Orleans
avg_stars_and_reviews_new_orleans = review_with_business[
    review_with_business['city'] == 'New Orleans'
].groupby('business_id').agg(
    avg_stars=('stars', 'mean'),
    number_of_reviews=('review_id', 'count')
)

# Query 10: Select top 5 categories by total checkins in New Orleans
top_checkins_categories_new_orleans = checkin_with_business[
    checkin_with_business['city'] == 'New Orleans'
].groupby('business_id').size().reset_index(name='total_checkins').merge(
    business_df[['business_id', 'categories']], on='business_id', how='left'
).groupby('categories')['total_checkins'].sum().nlargest(5)


# Business in Illinois
print("# Query 1: Business in IL:")
print(business_in_il)
print("\n")  # Adding a newline for better separation in the output

# Businesses with a star rating greater than 4
print("# Query 2: Business above 4 stars:")
print(business_above_4_stars)
print("\n")

# Five-star businesses in Illinois
print("# Query 3: Five star business in IL:")
print(five_star_business_in_il)
print("\n")

# Top 10 businesses in New Orleans by review count
print("# Query 4: Top business in New Orleans:")
print(top_business_in_new_orleans)
print("\n")

# Distinct categories of 5-star businesses in Illinois
print("# Query 5: Distinct categories in IL:")
print(distinct_categories_in_il)
print("\n")

# Top 5 businesses in New Orleans with more than 100 reviews
print("# Query 6: Top business over 100 reviews in New Orleans:")
print(top_business_over_100_reviews)
print("\n")

# Top 5 categories in New Orleans with more than 4.5 stars
print("# Query 7: Top categories over 4.5 stars in New Orleans:")
print(top_categories_over_45_stars)
print("\n")

# Average review count per category in New Orleans
print("# Query 8: Average review count per category in New Orleans:")
print(average_review_count_new_orleans)
print("\n")

# Average star rating and number of reviews for businesses in New Orleans
print("# Query 9: Avg stars and number of reviews in New Orleans:")
print(avg_stars_and_reviews_new_orleans)
print("\n")

# Top 5 categories by total check-ins in New Orleans
print("# Query 10: Top check-ins categories in New Orleans:")
print(top_checkins_categories_new_orleans)




# Query 1: Business in IL:
                   business_id                                 name  \
38      LcAozWCMLGjwRbokaJAKMg       Edwardsville Children's Museum   
41      ljxNT9p0y7YMPx0fcNBGig  Tony's Restaurant & 3rd Street Cafe   
48      bCBPXIVfVzBZBEpFu29dcg                      All In Shipping   
86      sE6jSnvMts_MAn-b4OkMAw                       K-9 Groom Room   
102     EuRGgOwJ0g1vTj2R04j37Q                          Crafty Crab   
...                        ...                                  ...   
150020  H9I0agL8A8r1eBh6p66o_Q                            Pizza Hut   
150042  cdbEEL_0TLOPgdfsB3PUvA                               Subway   
150110  eQ83iTisgPFzJ1YBO3p-IA                       Domino's Pizza   
150155  yKgxtgSBkDeuBsfjQVV-jA                       Chinese Dragon   
150344  mtGm22y5c2UHNXDFAjaPNw             Cyclery & Fitness Center   

                            address              city state postal_code  \
38                  722 Holyoake Rd      Edwa